# 01.Snomed2Vec.
## Fase 1 - Generación modelo Snomed2Vec.
###  Corpus iniciales, Informes de Alta del Servicio de Urgencias, Wikipedia y Snomed-CT
**Autor:** Ignacio Martinez Soriano. Data Scientist. Hospital Universitario "Rafael Mendez" & Medlab Media Group.

**Coautores:** 
* Juan Luis Castro Peña. Dept Ciencias de la computacion e Inteligencia Artificial. Universidad de Granada.
* Jesualdo Fernandez Breis. Dept. Informática y Sistemas. Universidad de Murcia.
* Ignacio San Román. Dept. Chief Artificial Intelligence Officer . Medlab Media Group.
* Adrian Alonso Barriuso. Research Technical Lead. Medlab Media Group.

**Fecha:** 28/01/2019.<br>
### Introducción Resumen:
Aplicamos la libreria gensim para generar varios modelos Word2Vec (Skip-Gram).<br>
Se genera los modelos, base, con las palabras vectorizadas de todos los textos.<br>
Se prepara los datos para generar modelos vectorizados.

### Datos de Entrada:
* /DataWork/Corpus-101.txt (Corpus y tokens de todos los Informes de Alta de Urgencias).
* /DataWork/wikipediaSpanish-2019.txt
* /Snomed-CT/Diccionario/Snomed-CorpusTerm-Jer.txt, Códigos Snomed Activos, con su jerarquia y Descripción.

### Productos generados, finales:

1. Modelo con los Datos de Inf-Alta + Descripciones Snomed. Sin pre-trained: **w2v-SK-s300-w5-m2-Total-Key.txt**
1. Modelo con los Datos de Wikipedia español + Descripciones Snomed-CT:

Utilizando los modelos anterioes, generamos los espacios vectoriales finales, aplicando el modelo a la descripción de los términos de Snomed-CT.

* Un registro para cada concepto con la siguiente estructura:<br>
    **|Id-Concept | TokensDescripcion | Jerarquia | [Vector de la Descripción]|**
    
    El Vector descripción está compuesto, según el modelo del dominio elegido (general o específico):
    * $ Descripcion (d) = (w_1,w_2,...,w_n), V[d] = \sum_{i=1}^{n}v[x_{i}]=v[x_{1}]+v[x_{2}]+...+v[x_{n}] $

## A new Approach Snomed-CT with word embedding - Snomed2Vec - Model.

### 1. Explicación y métodos:
Creación de un nuevo enfoque **Snomed2Vec**<br>
Se ha diseñado una herramienta para obtener los conceptos clínicos dentro del texto libre de cualquier informe médico. Asociándole el código mas aproximado, del concepto Snomed-CT. Utilizando **"Word Embeddings"** Devolviendo la lista de conceptos Snomed-CT, que aparecen en una frase y sus relaciones con otras entidadades clínicas.<br>

Nos basamos en dos artículos incialmente para la generación del modelo de espacio vectorial:<br>
1. [DNER Clinical (named entity recognition) from free clinical text to Snomed-CT concept](http://www.wseas.org/multimedia/journals/computers/2017/a205805-078.pdf). Ignacio Martinez Soriano. Juan Luis castro Peña. WSEAS Transactions on Computers, ISSN / E-ISSN: 1109-2750 / 2224-2872, Volume 16, 2017, Art. 10, pp. 83-91.
1. [STMC: Semantic Tag Medical Concept Using Word2Vec Representation](https://ieeexplore.ieee.org/document/8417270) Ignacio Martinez Soriano, Juan Luis Castro Peña. 2018 IEEE 31st International Symposium on Computer-Based Medical Systems (CBMS). Karsltad (Sweden).

Basándonos en la representación de la ontología para poder aplicarle **"Word2Vec"**, generamos una matriz base, con la siguiente estructura:<br>
**| Id-Concept | Jerarquia | Descripcion | [Vector de la Descripción] |** (La Jerarquía corresponde al nivel superior de las jerarquías existentes en la ontología de Snomed-CT, pudiendo así identificar que tipo de concepto clínico estamos utilizando.

Nos basamos en los siguientes artículos:
1. [Onto2Vec: joint vector-based representation of biological entities and their ontology-based annotations] (https://arxiv.org/abs/1802.00864), con el acceso al código github: https://github.com/bio-ontology-research-group/onto2vec
1. La mejora de la versión onto2vec, OPa2vec: [OPA2Vec: combining formal and informal content of biomedical ontologies to improve similarity-based prediction](https://arxiv.org/abs/1804.10922) y su acceso al código en github: https://github.com/bio-ontology-research-group/opa2vec/


Desarrollo de un **Clinical Semantic TAG** (Clinical Named Entity Recognition - NER).<br>
Generamos un espacio vectorial de palabras embebidas, "Word Embedding", utilizamos un enfoque híbrido con dos versiones, creando distintos enfoques:
* los Informes de Alta del Servicio de Urgencias del Hospital "Rafael Méndez", desde el año 2009-2016, mas las descripciones de los conceptos de Snomed-CT.
* Un modelo genarado con la descarga normalizada de Wikipedia en español (Fecha 01 de Octubre 2018) + las descripciones de los conceptos de Snomed.CT.

Para generar el espacio vectorial final de palabras embebidas, utilizamos nuestro modelo entrenado con las palabras de un gran corpus (Informes de Alta o Wikipedia) y lo volvemos a entrenar añadiendole las descripciones de Snomed-CT, generando de esta forma los vectores finales de nuestro Modelo.

**Generación Modelo final Snomed2Vec:**

Basándonos en los vectores de ese Modelo, aplicamos los vectores de las palabras para generar el Vector de la Descripción de Snomed-CT:<br>
Utilizamos el Modelo generado con Word2Vec, y lo aplicamos a la descripción de Snomed-CT, generando el siguiente vector asociado a la descripción **[d]**<br>
Siendo $ V(w_i) $ el Vector asociado a la palabra $ w_i $, según el modelo Word2Vec $ M $.<br>
* $ Descripcion-Snomed-CT(d) = (w_1,w_2,...,w_n),=> V[d] = \sum_{i=1}^{n}v[w_{i}]=v[w_{1}]+v[w_{2}]+...+v[w_{n}] $

El Modelo **Snomed2Vec** final: quedaria así:

* Un registro para cada concepto con la siguiente estructura:<br>
**| Id-Concept | Jerarquia | Descripcion | [Vector de la Descripción] |**

### Métodos implementados:

Implementamos una version de **Most_Similar()** para obtener los Conceptos similares de  **Snomed-CT**<br>

* **Mas_Similar('frase','clase mas proxima', numero-conceptos)**
*  Entrada: **frase**, **clase mas proxima**, se refiere a si se quiere buscar, los conceptos mas cercanos, según la jerarquía de Snomed-CT, El número de conceptos mas próxsimos para devolver un resultado.
*  Salida: (Lista de conceptos, nombrados en la **frase**, proporcionando los mas cercanos según clase buscada)

## 2. Snomed-CT:
Utilizamos la Ontología **Snomed-CT**, como Terminología clínica,para identificar los conceptos y codificarlos.<br>

### Estructura Lógica:
![Estructura Lógica Snomed-CT](./imagesNB/Modelo_logico_Snomed_1.png "Estructura Lógica")
*Imagen de https://confluence.ihtsdotools.org/*

**Conceptos:**<br>
Cada concepto representa un significado clínico único, al que se referencia con un identificador numérico único de SNOMED CT.

**Descripciones:**<br>
A cada concepto se asigna un conjunto de descripciones textuales. Éstas constituyen la forma legible de un concepto. Se utilizan dos tipos de descripciones para representar cada concepto - Descripción completa (FSN, por Fully specified name en inglés) y Sinónimo.

**Relaciones:**<br>
Una relación representa una asociación entre dos conceptos. Las relaciones se utilizan para definir lógicamente el significado de un concepto de manera que pueda procesarlo un ordenador. Un tercer concepto, denominado el tipo de relación (o atributo), se utiliza para representar el significado de la asociación entre el concepto de origen y el de destino. En SNOMED CT existen diferentes tipos de relaciones.

**Jerarquías:**<br>
Nivel superior con una breve descripción del contenido representado en su rama de la jerarquía.

>**|Hallazgo clínico|** representa el resultado de una observación, una evaluación o un juicio clínico e incluye a los conceptos utilizados para representar diagnósticos.<br>
**|Procedimiento|** representa actividades que se llevan a cabo durante la atención de la salud.<br>
**|Situación con contexto explícito|** representa hallazgos clínicos y procedimientos que aún no han ocurrido, (por ejemplo,|antecedente de infarto de miocardio|).<br>
**|Entidad observable|** representa una pregunta o una evaluación de las que se puede obtener una respuesta o un resultado (por ejemplo, |presión arterial sistólica|, |color del iris|, |género|). <br>
**|Estructura corporal|** representa estructuras anatómicas normales y anormales.<br>
**|Organismo|** representa organismos relevantes para la medicina humana y veterinaria.<br>
**|Sustancia|** representa sustancias en general, los constituyentes químicos de los productos farmacéuticos/biológicos, sustancias corporales, sustancias alimenticias y diagnósticas. <br>
**|Producto farmacéutico / biológico|** representa los productos farmacológicos (por ejemplo, |amoxicilina 250 mg, cápsula|, |paracetamol + codeína comprimido|). <br>
**|Espécimen|** representa entidades que se obtienen (por lo general del paciente) para realizar exámenes o análisis.<br>
**|Concepto especial|** representa conceptos que no desempeñan ningún papel en la lógica formal del modelo conceptual de la terminología, pero que pueden ser útiles para casos de uso específicos. <br>
**|Objeto físico|** representa objetos físicos naturales y fabricados por el hombre.<br>
**|Fuerza física|** representa fuerzas físicas que pueden desempeñar un papel como mecanismos de lesión (por ejemplo |fricción|, |radiación|, |corriente alterna|).<br>
**|Evento|** representa acontecimientos, con exclusión de procedimientos e intervenciones, (por ejemplo, |terremoto|). <br>
**|Ambiente o localización geográfica|** representa tipos de ambientes, así como lugares con nombres propios, como países, estados y regiones.<br>
**|Contexto social|** representa condiciones sociales y circunstancias significativas para la atención de la salud.<br>
**|Estadificaciones y escalas|** representa escalas de evaluación y sistemas de estadificación tumoral. <br>
**|Calificador|** representa los valores para algunos de los atributos de SNOMED CT, cuando esos valores no son subtipos de otros conceptos de nivel superior (por ejemplo, |izquierdo|, |resultado anormal|, |severo|).<br>
**|Elemento de registro|** representa contenido creado para brindar información a otras personas sobre eventos de registro o estado de asuntos (por ejemplo, |registro llevado por el paciente|, entrada de registro|, |sección correspondiente a antecedentes familiares|).<br>
**|Componente del modelo de SNOMED CT|** contiene los metadatos que se utilizan para la publicación de SNOMED CT. <br>


### Estructura General:
![Estructura General Modelo Snomed-CT](./imagesNB/Diseno.png "Modelo conceptual")
*Imagen de https://confluence.ihtsdotools.org/*

## 3. Generación de los Modelos de Espacio Vectorial, para palabras:

Para generar el modelo vectorial de palabras embebidas utilizamos Word2Vec, generado por Mikolov et al.<br>
>Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. 2013a. [Efficient estimation of word representations in vector space.](https://arxiv.org/abs/1301.3781) In Proceedings of ICLR Workshop


**Word2Vec**, es un framework semántico que utiliza una red neuronal superficial de una sola capa oculta, para aprender la representación de la palabra según su contexto.<br>
La idea es que el significado de las palabras viene definido, por el contexto. Esta idea fue desarrollada por Mikolov et al. generando una representacion distribuida de palabras en un espacio vectorial. Permitiendo la agrupación de palabras similares.
Para generar los Modelos del espacio vectorial de palabras, utilizamos la libreria Python **gensim:**<br>
>[*REHUREK, R., AND SOJKA, P. Software framework for topic modelling with large corpora. New Challenges For NLP Frameworks Programme (May 2010), 45–50.*](https://github.com/RaRe-Technologies/gensim)

Los parámetros utilizados para entrenar la red neuronal, han sido:
* El modelo elegido ha sido Skip-Gram.
* Tamaño del vector. size=300.
* Tamaño del a ventana de contexto. Window=8
* Mínimas palabras de estudio. min_count=1

**Se ha generado los siguientes Modelos:**

1. Modelo Dominio local, Word2Vec: Snomed2Vec-IA<br>
Se genera un modelo de espacio vectorial, con **Word2Vec**, basado en los **Informes de Alta de urgencias**. Con los siguientes parámetros:<br>
***(SkipGram, vector size 300, windows= 5, Negative=0, min=2)***

1. Modelo Dominio General, Word2Vec: Snomed2Vec-Wiki<br>
Se genera un modelo de espacio vectorial, con **Word2Vec**, basado en **Wikipedia Español**. Con los siguientes parámetros:<br>
***(SkipGram, vector size 300, windows= 5, Negative=0, min=2)***

1. Modelo Dominio General, Word2Vec: Snomed2Vec-CT<br>
Se genera un modelo de espacio vectorial, con **Word2Vec**, basado en **Descripciones del Snomed-CT**. Con los siguientes parámetros:<br>
***(SkipGram, vector size 300, windows= 5, Negative=0, min=2)***

1. Modelo Dominio local, ampliado con las descripciones de Snomed-CT:<br>
Se genera un modelo de espacio vectorial, con **Word2Vec**, basado en los Informes de Alta de urgencias y se vuelve a entrenar con las descripciones de Snomed-CT.

1. Modelo Dominio general, Wikipedia, ampliado con descripciones Snomed-CT:<br>
Se genera un modelo de espacio vectorial, con **Word2Vec**, basado en los datos de la wikipedia en español, ampliada con las descripciones de Snomed-CT.

**Modelo Final:**
![Snomed2Vec-Model02](./imagesNB/Snomed2Vec-Model02.png "Modelo Snomed2Vec")

## 4. Generación Vector para las frases:

Una vez generado el modelo elegido, se aplica a las descripciones de Snomed-CT, utilizando un DataSet construido específicamente para este framework.

El DataSet construido está compuesto por los siguientes campos **(id-Concepto, Jerarquia, descripción Concepto)**
Para identificar que tipo de concepto estamos seleccionando, se ha utilizado la jerarquia superior del concepto de Snomed-CT, al que pertenece.

**Generación Modelo final:**<br>
Utillizando los modelos anteriores, se aplica a la descripción del concepto, para generar el vector de la descripción:

Siendo $ V(w_i) $ el Vector asociado a la palabra $ w_i $, según el modelo Word2Vec $ M $.<br>
* $ Descripcion-Snomed-CT(d) = (w_1,w_2,...,w_n),=> V[d] = \sum_{i=1}^{n}v[w_{i}]=v[w_{1}]+v[w_{2}]+...+v[w_{n}] $

Obteniendo un modelo final **Snomed2Vec**, compuesto por:
**(id-Concept | Jerarquia | Descripción | vecSnom(d) )**:
![Estructura Snomed2Vec](./imagesNB/SnomedWork.png "Modelo Snomed2Vec")


## 5. Métodos,  cálculo similaridad y mas_similar():<br>
Para calcular los conceptos similares, se aplica la distancia del coseno, entre los vectores de cada palabra o frase:<br>
![Distancia Coseno Similaridad](./imagesNB/cosine-similarity.png "Modelo Snomed2Vec")

### Snomed2Vec.Similar(texto1, texto2):<br>
**Entrada:** texto1=frase secuencia palabras<br>
**Sallida:** grado de similaridad = distancia del coseno, entre los vectores de V[texto1] y v[texto2]<br>

### Snomed2Vec.mas_similar(texto,clase):<br>
**Entrada:**<br>
*texto*=frase secuencia palabras<br>
*clase*= Tipo de concepto, para sacar los más proximos, respecto a la entrad de texto.<br>
**Salida:** grado de similaridad = distancia del coseno, entre los vectores de V[texto] y v[Todas Desc Snomed-CT]<br>


## 6. Resultados de la investigación:
* DataSet de Snomed-CT, compuesto por los conceptos con su jerarquia superior, y sus sinónimos.
* Modelo Wikipedia en Español, entrenado con word2vec y normalizado. (palabras en minúsculas). Tamaño 300 y con Skip-Gram.
* Modelos Snomed2Vec, entrenados, con datos específicos de un dominio local, (Informes de Alta Urgencias + Desc. Terminos de Snomed-CT).
* framework, para obtner los conceptos y relaciones, según una frase de entrada. (Por ejemplo un diagnóstico principal).
* página web para poder testear y probar el framework.


  
### Representation Learning:
I create some jupyter notebook to test the framewotk
Prueba
  ```
  python Snomed2Vec.py *"sentence test"*
  ```
  This script get the SNomed-CT Clinical concept from a free text sentence. 

## 7. Data sources, Space vector Models:
** Corpus Gold:**

## Final notes:
This documentation is part of my PH.D.Thesis and share to improve this new approach.
For any comments or help needed with how to test *Snomed2Vec* tool, you can write to: ignacio.martinez@carm.es



In [60]:
# NLP Informe de Alta - carga de librerias- Modelo W2v
import sys
#p = r'D:\Doctorado\codigo'
p='/media/nacho/DatosHD/Doctorado/CBMS2019/codigo'
sys.path.append(p)

import pandas as pd
import numpy as np
from numpy import exp, log, dot, zeros, outer, random, dtype, float32 as REAL,\
    double, uint32, seterr, array, uint8, vstack, fromstring, sqrt, newaxis,\
    ndarray, empty, sum as np_sum, prod, ones, ascontiguousarray
import logging
import IPython.display as disp
from gensim.models import Word2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
pd.set_option('max_colwidth', 100) #para que se impriman las columnas con mayor anchura en el notebook

## Creación Modelo 01 - Dominio local (InfAlta + Descr Snomed-CT):
### Carga del Corpus-101.txt.
Utilizamos el Corpus de los Informes de Alta de Urgencias del Hospital "Rafael Méndez" (Años 2009-2016), sin los signos de puntuación y con los datos normalizados **(minúsculas)** y **sin acentos**.<br>
Corpus utilizado **Corpus-101.txt**

In [2]:
#Carga de los datos Corpus normalizado, Informe de Alta separados en tokens normalizados.
fich = '/DataWork/Corpus-101.txt'
f = p+fich
df = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')

In [3]:
df.head()

,corpus,Token
0,obstruccion de colon hta dm en tto con ado no dlp anemia en tto con fe ait hace un año sin s...,"['obstruccion', 'colon', 'hta', 'dm', 'tto', 'ado', 'no', 'dlp', 'anemia', 'tto', 'fe', 'ait', '..."
1,derrame pleural no raf no dm hta no dlp estenosis aortica degenerativa cardiopatia hiperten...,"['derrame', 'pleural', 'no', 'raf', 'no', 'dm', 'hta', 'no', 'dlp', 'estenosis', 'aortica', 'deg..."
2,insuficiencia respiratoria de probable origen mixto acxfa de respuesta ventricular rapida exit...,"['insuficiencia', 'respiratoria', 'probable', 'origen', 'mixto', 'acxfa', 'respuesta', 'ventricu..."
3,gastritis aguda padres sanos 2 tios maternos muerte subita por iam embarazo controlado de cur...,"['gastritis', 'aguda', 'padres', 'sanos', '2', 'tios', 'maternos', 'muerte', 'subita', 'iam', 'e..."
4,scasest padres fallecidos de vejez no raf hta no dm no dlp depresion hace 20 años ningun...,"['scasest', 'padres', 'fallecidos', 'vejez', 'no', 'raf', 'hta', 'no', 'dm', 'no', 'dlp', 'depre..."


In [4]:
# Convertir los datos cargados en listas de texto, despues de cargar los datos.
from ast import literal_eval
df['Token'] = df['Token'].apply(literal_eval)

In [5]:
df.head()

,corpus,Token
0,obstruccion de colon hta dm en tto con ado no dlp anemia en tto con fe ait hace un año sin s...,"[obstruccion, colon, hta, dm, tto, ado, no, dlp, anemia, tto, fe, ait, hace, año, secuelas, no, ..."
1,derrame pleural no raf no dm hta no dlp estenosis aortica degenerativa cardiopatia hiperten...,"[derrame, pleural, no, raf, no, dm, hta, no, dlp, estenosis, aortica, degenerativa, cardiopatia,..."
2,insuficiencia respiratoria de probable origen mixto acxfa de respuesta ventricular rapida exit...,"[insuficiencia, respiratoria, probable, origen, mixto, acxfa, respuesta, ventricular, rapida, ex..."
3,gastritis aguda padres sanos 2 tios maternos muerte subita por iam embarazo controlado de cur...,"[gastritis, aguda, padres, sanos, 2, tios, maternos, muerte, subita, iam, embarazo, controlado, ..."
4,scasest padres fallecidos de vejez no raf hta no dm no dlp depresion hace 20 años ningun...,"[scasest, padres, fallecidos, vejez, no, raf, hta, no, dm, no, dlp, depresion, hace, 20, años, n..."


### Carga de Datos de Snomed-CT.
Se carga las descripciones de los términos de Snomed-CT, para generar el corpus, final.

In [7]:
# Carga de Datos para el Trabajo final:
fich = '/DataWork/Snomed-CorpusTerm-Jer.txt'
f = p+fich

dfSnoWork = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')

In [9]:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia
0,102002,"['hemoglobina', 'okaloosa']",Sustancia
1,102002,"['hb', '48', 'cd7', 'leu', 'arg']",Sustancia
2,103007,"['virus', 'fibroma', 'ardillas']",Organismo
3,104001,"['escision', 'lesion', 'rotula']",Procedimiento
4,104001,"['escision', 'local', 'lesion', 'o', 'tejido', 'rotula']",Procedimiento


In [10]:
# Convertir los datos cargados en listas de texto, despues de cargar los datos.
from ast import literal_eval
dfSnoWork['corpusTerm'] = dfSnoWork['corpusTerm'].apply(literal_eval)
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia
0,102002,"[hemoglobina, okaloosa]",Sustancia
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia
2,103007,"[virus, fibroma, ardillas]",Organismo
3,104001,"[escision, lesion, rotula]",Procedimiento
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento


In [12]:
print('Tamaño del dataframe dfSnoWork = 516211, para la gestion de los códigos de Snomed: %d' % len(dfSnoWork))

Tamaño del dataframe dfSnoWork = 516211, para la gestion de los códigos de Snomed: 516211


### Unifico los dos Dataframes, para pobtener el corpus definitivo.
Corpus final, con los Informes de Alta + Descripciones Términos de Snomed-CT

In [13]:
cols=['Token']
df[cols]

,Token
0,"[obstruccion, colon, hta, dm, tto, ado, no, dlp, anemia, tto, fe, ait, hace, año, secuelas, no, ..."
1,"[derrame, pleural, no, raf, no, dm, hta, no, dlp, estenosis, aortica, degenerativa, cardiopatia,..."
2,"[insuficiencia, respiratoria, probable, origen, mixto, acxfa, respuesta, ventricular, rapida, ex..."
3,"[gastritis, aguda, padres, sanos, 2, tios, maternos, muerte, subita, iam, embarazo, controlado, ..."
4,"[scasest, padres, fallecidos, vejez, no, raf, hta, no, dm, no, dlp, depresion, hace, 20, años, n..."
5,"[vertigo, paciente, 78, años, consulta, mareo, tipo, inestabilidad, aproximadamente, 1, mes, evo..."
6,"[politraumatizado, politraumatizado]"
7,"[gea, no, hta, no, dlp, no, dm, neo, colon, hace, 12, años, alzheimer, ceguera, varios, años, ge..."
8,"[politraumatizado, no, raf, tendones, rodilla, izquierda, muñeca, izquierda, luxacion, cadera, i..."
9,"[tep, insuficiencia, renal, cronica, alergica, penicilina, hta, diabetes, mellitus, tipo, ii, tr..."


In [23]:
corpus = df['Token'].append(dfSnoWork['corpusTerm'], ignore_index=True)

In [30]:
len(corpus)

1152650

## 1. Creación Modelo utilizando Word2Vec -------- Modelo Skip-Gram -----------.
Parámetros utilizados:<br>
* text = **corpus_token** Secuencia de tokens normalizados
* window = 8 tamaño de la ventana (SkipGram)
* size = 300 tamaño del vector, que contiene la capa oculta aprendida.
* workers=4 threads activos
* min_count= 1 Mínima frecuencia por token, para filtrar palabras raras.
* sample= 0.05 peso de palabras comunes
* sg = 1 (sg = 0 para cbow y sg = 1 para Skip-gram)
* iter = 5,
* hs = 0

In [31]:
# Entrenar el Modelo con los Informes de Alta + las Descripciones de Snomed-CT:

model = Word2Vec(corpus,
                size=300,
                window=8,
                workers=4,
                min_count=1,
                sample=0.05,
                sg=1,
                iter=5,
                hs=0)

2019-01-28 11:16:03,309 : INFO : collecting all words and their counts
2019-01-28 11:16:03,310 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-28 11:16:03,578 : INFO : PROGRESS: at sentence #10000, processed 2088374 words, keeping 63789 word types
2019-01-28 11:16:03,824 : INFO : PROGRESS: at sentence #20000, processed 4011185 words, keeping 93182 word types
2019-01-28 11:16:04,052 : INFO : PROGRESS: at sentence #30000, processed 5795240 words, keeping 115771 word types
2019-01-28 11:16:04,271 : INFO : PROGRESS: at sentence #40000, processed 7518644 words, keeping 132646 word types
2019-01-28 11:16:04,507 : INFO : PROGRESS: at sentence #50000, processed 9322445 words, keeping 150560 word types
2019-01-28 11:16:04,730 : INFO : PROGRESS: at sentence #60000, processed 11022147 words, keeping 166372 word types
2019-01-28 11:16:04,959 : INFO : PROGRESS: at sentence #70000, processed 12755078 words, keeping 180206 word types
2019-01-28 11:16:05,173 : INFO :

2019-01-28 11:16:15,204 : INFO : PROGRESS: at sentence #710000, processed 84893627 words, keeping 497638 word types
2019-01-28 11:16:15,224 : INFO : PROGRESS: at sentence #720000, processed 84927535 words, keeping 499246 word types
2019-01-28 11:16:15,239 : INFO : PROGRESS: at sentence #730000, processed 84961883 words, keeping 500842 word types
2019-01-28 11:16:15,255 : INFO : PROGRESS: at sentence #740000, processed 84995875 words, keeping 502406 word types
2019-01-28 11:16:15,268 : INFO : PROGRESS: at sentence #750000, processed 85029856 words, keeping 503858 word types
2019-01-28 11:16:15,289 : INFO : PROGRESS: at sentence #760000, processed 85063831 words, keeping 505281 word types
2019-01-28 11:16:15,307 : INFO : PROGRESS: at sentence #770000, processed 85098614 words, keeping 506596 word types
2019-01-28 11:16:15,321 : INFO : PROGRESS: at sentence #780000, processed 85137793 words, keeping 507317 word types
2019-01-28 11:16:15,334 : INFO : PROGRESS: at sentence #790000, processe

2019-01-28 11:16:49,939 : INFO : PROGRESS: at 0.13% examples, 66233 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:16:51,088 : INFO : PROGRESS: at 0.14% examples, 66334 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:16:52,233 : INFO : PROGRESS: at 0.15% examples, 66443 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:16:53,372 : INFO : PROGRESS: at 0.15% examples, 66552 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:16:54,526 : INFO : PROGRESS: at 0.16% examples, 66627 words/s, in_qsize 6, out_qsize 1
2019-01-28 11:16:55,654 : INFO : PROGRESS: at 0.17% examples, 66748 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:16:56,757 : INFO : PROGRESS: at 0.17% examples, 66909 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:16:57,900 : INFO : PROGRESS: at 0.18% examples, 66982 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:16:59,039 : INFO : PROGRESS: at 0.19% examples, 67039 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:17:00,181 : INFO : PROGRESS: at 0.20% examples, 67123 words/s, in_qsize 7, ou

2019-01-28 11:18:23,338 : INFO : PROGRESS: at 0.75% examples, 68605 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:24,392 : INFO : PROGRESS: at 0.76% examples, 68496 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:25,542 : INFO : PROGRESS: at 0.76% examples, 68499 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:26,690 : INFO : PROGRESS: at 0.77% examples, 68505 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:27,822 : INFO : PROGRESS: at 0.78% examples, 68513 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:28,968 : INFO : PROGRESS: at 0.79% examples, 68514 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:30,118 : INFO : PROGRESS: at 0.79% examples, 68518 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:31,289 : INFO : PROGRESS: at 0.80% examples, 68509 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:32,443 : INFO : PROGRESS: at 0.81% examples, 68507 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:18:33,581 : INFO : PROGRESS: at 0.82% examples, 68515 words/s, in_qsize 7, ou

2019-01-28 11:19:56,943 : INFO : PROGRESS: at 1.40% examples, 68708 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:19:58,087 : INFO : PROGRESS: at 1.41% examples, 68711 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:19:59,221 : INFO : PROGRESS: at 1.42% examples, 68717 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:20:00,359 : INFO : PROGRESS: at 1.42% examples, 68722 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:20:01,491 : INFO : PROGRESS: at 1.43% examples, 68727 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:20:02,614 : INFO : PROGRESS: at 1.44% examples, 68734 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:20:03,733 : INFO : PROGRESS: at 1.45% examples, 68743 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:20:04,878 : INFO : PROGRESS: at 1.46% examples, 68744 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:20:06,027 : INFO : PROGRESS: at 1.46% examples, 68747 words/s, in_qsize 7, out_qsize 0
2019-01-28 11:20:07,172 : INFO : PROGRESS: at 1.47% examples, 68749 words/s, in_qsize 7, ou

...

2019-01-28 12:52:14,936 : INFO : PROGRESS: at 87.13% examples, 70256 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:16,091 : INFO : PROGRESS: at 87.14% examples, 70256 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:17,240 : INFO : PROGRESS: at 87.15% examples, 70256 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:18,378 : INFO : PROGRESS: at 87.16% examples, 70256 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:19,498 : INFO : PROGRESS: at 87.17% examples, 70256 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:20,631 : INFO : PROGRESS: at 87.18% examples, 70255 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:21,752 : INFO : PROGRESS: at 87.19% examples, 70255 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:22,885 : INFO : PROGRESS: at 87.20% examples, 70255 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:24,006 : INFO : PROGRESS: at 87.21% examples, 70255 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:52:25,133 : INFO : PROGRESS: at 87.22% examples, 70255 words/s, in_q

2019-01-28 12:53:46,821 : INFO : PROGRESS: at 87.97% examples, 70244 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:47,965 : INFO : PROGRESS: at 87.98% examples, 70244 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:49,104 : INFO : PROGRESS: at 87.99% examples, 70244 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:50,249 : INFO : PROGRESS: at 88.00% examples, 70244 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:51,390 : INFO : PROGRESS: at 88.01% examples, 70244 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:52,529 : INFO : PROGRESS: at 88.02% examples, 70244 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:53,654 : INFO : PROGRESS: at 88.03% examples, 70244 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:54,800 : INFO : PROGRESS: at 88.04% examples, 70243 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:55,936 : INFO : PROGRESS: at 88.05% examples, 70243 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:53:57,069 : INFO : PROGRESS: at 88.06% examples, 70243 words/s, in_q

2019-01-28 12:55:18,713 : INFO : PROGRESS: at 88.84% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:19,854 : INFO : PROGRESS: at 88.86% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:20,994 : INFO : PROGRESS: at 88.87% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:21,996 : INFO : PROGRESS: at 88.88% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:23,011 : INFO : PROGRESS: at 88.89% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:24,149 : INFO : PROGRESS: at 88.91% examples, 70232 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:25,272 : INFO : PROGRESS: at 88.93% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:26,390 : INFO : PROGRESS: at 88.94% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:27,512 : INFO : PROGRESS: at 88.96% examples, 70233 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:55:28,646 : INFO : PROGRESS: at 88.97% examples, 70233 words/s, in_q

2019-01-28 12:56:50,065 : INFO : PROGRESS: at 89.87% examples, 70227 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:51,208 : INFO : PROGRESS: at 89.89% examples, 70227 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:52,336 : INFO : PROGRESS: at 89.90% examples, 70227 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:53,457 : INFO : PROGRESS: at 89.91% examples, 70227 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:54,596 : INFO : PROGRESS: at 89.92% examples, 70226 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:55,731 : INFO : PROGRESS: at 89.93% examples, 70226 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:56,869 : INFO : PROGRESS: at 89.94% examples, 70226 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:58,007 : INFO : PROGRESS: at 89.95% examples, 70226 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:56:59,124 : INFO : PROGRESS: at 89.96% examples, 70226 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:57:00,265 : INFO : PROGRESS: at 89.97% examples, 70226 words/s, in_q

2019-01-28 12:58:21,545 : INFO : PROGRESS: at 90.75% examples, 70224 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:22,680 : INFO : PROGRESS: at 90.76% examples, 70224 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:23,828 : INFO : PROGRESS: at 90.77% examples, 70223 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:24,954 : INFO : PROGRESS: at 90.78% examples, 70223 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:26,101 : INFO : PROGRESS: at 90.79% examples, 70223 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:27,240 : INFO : PROGRESS: at 90.80% examples, 70223 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:28,367 : INFO : PROGRESS: at 90.81% examples, 70223 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:29,502 : INFO : PROGRESS: at 90.82% examples, 70223 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:30,635 : INFO : PROGRESS: at 90.84% examples, 70223 words/s, in_qsize 7, out_qsize 0
2019-01-28 12:58:31,781 : INFO : PROGRESS: at 90.85% examples, 70223 words/s, in_q

In [32]:
# Grabamos el modelo generado al disco. Preparado para estar Optimizado en Memoria - SIN posibilidad de Re-Entrenar
fich = '/Models/SKs300w8m1-InfAltSnomed.txt'
f = p+fich
model.wv.save_word2vec_format(f, binary=False)

2019-01-28 20:04:43,997 : INFO : storing 538631x300 projection weights into /media/nacho/DatosHD/Doctorado/CBMS2019/codigo/Models/Snomed2Vec-SKs300w8m1-InfAltSnomed.txt


In [42]:
print(model)

Word2Vec(vocab=538631, size=300, alpha=0.025)


In [40]:
#Ejemplo de utilizacion 
model.most_similar('dr', topn=20)

[('lopez', 0.7301999926567078),
 ('dra', 0.7116214632987976),
 ('mateo', 0.7096290588378906),
 ('navarro', 0.702811062335968),
 ('raul', 0.6763116121292114),
 ('guardia', 0.674852728843689),
 ('cano', 0.6733564138412476),
 ('jeronimo', 0.6603413820266724),
 ('quesada/', 0.6598756313323975),
 ('ruiz', 0.6595959663391113),
 ('villar', 0.6592674255371094),
 ('anton', 0.6577282547950745),
 ('ochoa', 0.6564529538154602),
 ('gonzalez', 0.6549363136291504),
 ('wilches', 0.6543387174606323),
 ('cerillo', 0.6533350944519043),
 ('rodriguez', 0.6523047685623169),
 ('buendia', 0.6485064625740051),
 ('cerrillo', 0.6467321515083313),
 ('antonio', 0.6430599689483643)]

In [37]:
# 01 - Pruebas Modelo. Solo Informes de ALta:
model.most_similar('izda', topn=10)

[('dcha', 0.7724915146827698),
 ('izdo', 0.6958502531051636),
 ('zida', 0.693569540977478),
 ('dercha', 0.6804987192153931),
 ('zda', 0.6766991019248962),
 ('izd', 0.6584492921829224),
 ('derecha', 0.6486457586288452),
 ('iza', 0.6147016286849976),
 ('dereccha', 0.6140893697738647),
 ('dhca', 0.6056210398674011)]

## 2. Estructura espacio Vectorial final Snomed2Vec-Model:
Se propone la siguiente esctructura de la Matriz, Snomed2Vec.

Propuesta:

* Un registro para cada concepto con la siguiente estructura:<br>
    **|Id-Concept | TokensDescripcion | Jerarquia | [Vector de la Descripción]|**
    
    El Vector descripción está compuesto, según el modelo del dominio elegido (general o específico):
    * $ Descripcion (d) = (w_1,w_2,...,w_n), V[d] = \sum_{i=1}^{n}v[x_{i}]=v[x_{1}]+v[x_{2}]+...+v[x_{n}] $

In [38]:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia
0,102002,"[hemoglobina, okaloosa]",Sustancia
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia
2,103007,"[virus, fibroma, ardillas]",Organismo
3,104001,"[escision, lesion, rotula]",Procedimiento
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento


In [51]:
def creaVecSnomedOpci4(row):

    desc=row.corpusTerm
    cod = row.idConcept
    
    output=[]
    buffer=np.zeros((300,), dtype=np.float32)
    for w in desc:
        try:
            buffer = buffer + model.wv.word_vec(w)
        except:   
            buffer = buffer + np.zeros((300,), dtype=np.float32)

    output.append(buffer)   
    
    return output

In [52]:
# Creacion Modelo Opcion 04:
dfSnoWork['vecSnom']=dfSnoWork.apply(creaVecSnomedOpci4, axis=1)

In [53]:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia,vecSnom
0,102002,"[hemoglobina, okaloosa]",Sustancia,"[[0.503108, 0.466876, -0.0276602, -0.234006, -0.116602, 0.484161, 0.125, -0.351791, 0.168337, -0..."
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia,"[[-0.301794, -0.120972, 0.0392014, 0.748237, 0.531802, -0.969592, 0.567389, -1.43454, 0.598673, ..."
2,103007,"[virus, fibroma, ardillas]",Organismo,"[[1.08219, -0.860717, 0.460997, 0.467138, -0.109274, 0.35905, 0.0495269, -1.52099, 0.485955, -0...."
3,104001,"[escision, lesion, rotula]",Procedimiento,"[[0.956222, 0.32638, 0.567415, 0.278348, -0.473122, 1.32149, 0.381964, -0.338385, -0.179837, -0...."
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento,"[[1.09046, 1.19666, 0.137113, 1.16608, -1.54469, 1.88294, 0.745489, -1.53078, 0.679535, -0.65002..."


model.wv.word_vec('hemoglobina')+model.wv.word_vec('okaloosa')

** 5.2.1.1. Hacer unas adptaciones, para crear correctamente la columna vecSnom -> numpy.array(300,)**

In [61]:
def cambiaNumpy(row):
    vocab = row[0]
    voc = np.array(vocab, dtype=REAL)
    return voc

In [62]:
# Opcion 4. Actualizr vecSnom column
dfSnoWork['vecSnom'] = dfSnoWork['vecSnom'].apply(cambiaNumpy)

In [63]:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia,vecSnom
0,102002,"[hemoglobina, okaloosa]",Sustancia,"[0.503108, 0.466876, -0.0276602, -0.234006, -0.116602, 0.484161, 0.125, -0.351791, 0.168337, -0...."
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia,"[-0.301794, -0.120972, 0.0392014, 0.748237, 0.531802, -0.969592, 0.567389, -1.43454, 0.598673, -..."
2,103007,"[virus, fibroma, ardillas]",Organismo,"[1.08219, -0.860717, 0.460997, 0.467138, -0.109274, 0.35905, 0.0495269, -1.52099, 0.485955, -0.6..."
3,104001,"[escision, lesion, rotula]",Procedimiento,"[0.956222, 0.32638, 0.567415, 0.278348, -0.473122, 1.32149, 0.381964, -0.338385, -0.179837, -0.3..."
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento,"[1.09046, 1.19666, 0.137113, 1.16608, -1.54469, 1.88294, 0.745489, -1.53078, 0.679535, -0.650022..."


### 5.3. Grabamos el segundo modelo de Snomed2Vec. Modelo generado con los informes de ALta. Opcion.2:
**Snomed2Vec-SK300-IA-Op2.txt**
Se guarda el fichero en formato numpy. Se utiliza esta versión para almacenar correctamente los vectores:

In [64]:
# Prueba con fichero numpy. Guardar Se utiliza esta versión para almacenar correctamente los vectores:
fich = '/Models/Snomed2Vec-SK300-IA-Snomed.npy'
f = p+fich
cols=['idConcept','jerarquia','corpusTerm','vecSnom']
num_df = dfSnoWork[cols]

with open(f,'wb') as np_file:
    np.save(np_file, num_df)